In [1]:
import pandas as pd
import numpy as np
import tensorflow
import theano
import pickle

from pandas import *

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import *

from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.wrappers.scikit_learn import KerasClassifier
from keras import regularizers
from keras import backend

import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.externals.joblib import parallel_backend
from random import *

Using TensorFlow backend.


In [2]:
df = pd.read_csv('../data/df_for_model.csv')
# df['start_time'] = df['start_time']//1000
df = df[df['result'] != 0.5]

In [3]:
bins = list(range(-2000, -499, 50))
bins.extend(list(range(-490, -199, 10)))
bins.extend(list(range(-195, 196, 5)))
bins.extend(list(range(200, 501, 10)))
bins.extend(list(range(550, 2001, 50)))

In [4]:
df['diff_bin'] = pd.cut(df['diff'], bins=bins, labels=bins[:len(bins)-1])

In [5]:
numeric_predictors = ['color', 'diff_bin', 'game_time', 'start_time', 'weekday']
# X = df[numeric_predictors].values
X = df[numeric_predictors]
y = np.array(df['result'])

In [6]:
print(X.shape)
print(y.shape)

(2076, 5)

In [8]:
X = X.astype('float64')
y = y.astype('int64')

In [9]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15, random_state=6)

In [19]:
limi = randint(int(len(X)*.85), int(len(X)*.95))
limi

1901

In [20]:
X_train = X[:limi]
y_train = y[:limi]
X_test = X[limi:]
y_test = y[limi:]

In [21]:
X_train.shape

(1901, 5)

In [22]:
X_test.shape

(175, 5)

In [23]:
y_train.shape

(1901,)

In [24]:
std_sclr = StandardScaler()
X_train = std_sclr.fit_transform(X_train)
X_test = std_sclr.fit_transform(X_test)

In [25]:
# std_sclr.scale_

In [26]:
# def _classifier():
#     classifier = Sequential()

#     classifier.add(Dense(units=13, kernel_initializer='uniform',
#                          activation='relu', input_dim=5))

#     classifier.add(
#         Dense(units=83, kernel_initializer='uniform', activation='softmax'))

#     classifier.add(
#         Dense(units=13, kernel_initializer='uniform', activation='relu'))

# #     classifier.add(
# #         Dense(units=109, kernel_initializer='uniform', activation='softmax'))

# #     classifier.add(
# #         Dense(units=47, kernel_initializer='uniform', activation='relu'))

# #     classifier.add(
# #         Dense(units=11, kernel_initializer='uniform', activation='softmax'))

#     classifier.add(
#         Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))

#     classifier.compile(
#         optimizer='sgd', loss='binary_crossentropy', metrics=['hinge'])

#     return classifier

In [27]:
def dropout_classifier(optimizer, metrics):
    classifier = Sequential()
    classifier.add(Dense(units=5, kernel_initializer='uniform',
                         activation='relu', input_dim=5,
                         kernel_regularizer=regularizers.l2(0.01),
                         activity_regularizer=regularizers.l1(0.01)))

    classifier.add(Dropout(rate=0.16))

    classifier.add(
        Dense(units=29, kernel_initializer='uniform', activation='softmax',
              kernel_regularizer=regularizers.l2(0.01),
              activity_regularizer=regularizers.l1(0.01)))

    classifier.add(Dropout(rate=0.08))

    classifier.add(
        Dense(units=7, kernel_initializer='uniform', activation='relu'))

    classifier.add(Dropout(rate=0.04))

#     classifier.add(
#         Dense(units=11, kernel_initializer='uniform', activation='hard_sigmoid'))

    classifier.add(
        Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))

    classifier.compile(
        optimizer=optimizer, loss='binary_crossentropy', metrics=[metrics])

    return classifier

In [28]:
bs_1 = randint(10,20)
bs_2 = randint(20,30)
ep_1 = randint(100,200)
ep_2 = randint(200,300)

parameters = {'batch_size': [bs_1, bs_2],
              'nb_epoch': [ep_1, ep_2],
              'optimizer': ['Nadam', 'Adamax', 'RMSprop'],
              'metrics': ['accuracy', 'categorical_accuracy', 'mape']
              }

In [29]:
classifier = KerasClassifier(build_fn=dropout_classifier)

In [30]:
grid_search = GridSearchCV(classifier, parameters,
                           scoring='f1_micro', cv=5, n_jobs=-2)

In [31]:
print([bs_1, bs_2, ep_1, ep_2])
grid_search = grid_search.fit(X=X_train, y=y_train)

[15, 24, 112, 292]
Epoch 1/1
1520/1520 [==============================] - 1s 766us/step - loss: 0.8450 - acc: 0.4987
Epoch 1/1
1521/1521 [==============================] - 1s 792us/step - loss: 0.8446 - acc: 0.4885
Epoch 1/1
1521/1521 [==============================] - 1s 829us/step - loss: 0.8446 - acc: 0.4977
Epoch 1/1
1521/1521 [==============================] - 1s 953us/step - loss: 0.8450 - acc: 0.4878
Epoch 1/1
1521/1521 [==============================] - 1s 877us/step - loss: 0.8450 - acc: 0.5082
Epoch 1/1
1520/1520 [==============================] - 1s 853us/step - loss: 0.8453 - acc: 0.4842
Epoch 1/1
1521/1521 [==============================] - 1s 866us/step - loss: 0.8443 - acc: 0.5056
Epoch 1/1
1521/1521 [==============================] - 1s 886us/step - loss: 0.8448 - acc: 0.5043
Epoch 1/1
1521/1521 [==============================] - 1s 914us/step - loss: 0.8445 - acc: 0.4944
Epoch 1/1
1521/1521 [==============================] - 1s 930us/step - loss: 0.8451 - acc: 0.4694
E

KeyboardInterrupt: 

In [23]:
best_params = grid_search.best_params_
best_params

{'batch_size': 11,
 'metrics': 'accuracy',
 'nb_epoch': 136,
 'optimizer': 'Nadam'}

In [24]:
best_accu = grid_search.best_score_
best_accu

0.5

In [25]:
grid_search.cv_results_

/home/guess/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/guess/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/guess/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/guess/anaconda3/lib/python3.

{'mean_fit_time': array([1.95626278, 1.86280422, 2.04237003, 2.65089693, 2.50077305,
        2.76881981, 3.51045456, 3.13306255, 3.35499706, 3.9417161 ,
        3.77045817, 4.10874844, 4.59338212, 4.4006001 , 4.83682075,
        5.30044928, 5.13736792, 5.32125216, 5.8378161 , 5.51505432,
        5.84917669, 6.35840449, 6.23469224, 6.40556731, 6.94099956,
        6.91315556, 7.0387198 , 7.59981046, 7.75728326, 7.8697196 ,
        8.61835566, 7.92901921, 8.53506799, 8.75249362, 8.59890614,
        9.75399041]),
 'std_fit_time': array([0.0407468 , 0.09310205, 0.08139113, 0.10323613, 0.23038569,
        0.17639263, 0.24815985, 0.17819387, 0.1059951 , 0.24145006,
        0.22014575, 0.25223405, 0.11204237, 0.04961857, 0.29187926,
        0.23819855, 0.19555931, 0.19253297, 0.33804709, 0.38337115,
        0.19191702, 0.23797548, 0.53996041, 0.11535254, 0.06349024,
        0.18512041, 0.06298417, 0.13247473, 0.55428582, 0.16007322,
        0.20718979, 0.05266484, 0.21150767, 0.09837604, 0.062

In [26]:
y_pred = grid_search.predict(X_test)
y_pred = (y_pred > 0.5)

In [27]:
cm = confusion_matrix(y_test, y_pred)
cm

array([[106,   0],
       [108,   0]])

In [28]:
print(f'classification_report:\n{classification_report(y_test, y_pred)}')
print('---------------------------------------------------------------\n\n')
print(f'average_precision_score:\n{average_precision_score(y_test, y_pred)*100}%')
print('---------------------------------------------------------------\n\n')
print(f'f1_score:\n{f1_score(y_test, y_pred)*100}%')
print('---------------------------------------------------------------\n\n')
print(f'accuracy_score:\n{accuracy_score(y_test, y_pred)*100}%')
print('---------------------------------------------------------------\n\n')
print(f'cohen_kappa_score:\n{cohen_kappa_score(y_test, y_pred)}')
print('---------------------------------------------------------------\n\n')
print(f'roc_curve:\n{roc_curve(y_test, y_pred)}')
print('---------------------------------------------------------------\n\n')
print(f'roc_auc_score:\n{roc_auc_score(y_test, y_pred)*100}%')
print('---------------------------------------------------------------\n\n')
print(f'brier_score_loss:\n{brier_score_loss(y_test, y_pred)*100}%')
print('---------------------------------------------------------------\n\n')
print(f'recall_score:\n{recall_score(y_test, y_pred)*100}%')

classification_report:
              precision    recall  f1-score   support

           0       0.50      1.00      0.66       106
           1       0.00      0.00      0.00       108

   micro avg       0.50      0.50      0.50       214
   macro avg       0.25      0.50      0.33       214
weighted avg       0.25      0.50      0.33       214

---------------------------------------------------------------


average_precision_score:
50.467289719626166%
---------------------------------------------------------------


f1_score:
0.0%
---------------------------------------------------------------


accuracy_score:
49.532710280373834%
---------------------------------------------------------------


cohen_kappa_score:
0.0
---------------------------------------------------------------


roc_curve:
(array([0., 1.]), array([0., 1.]), array([1, 0]))
---------------------------------------------------------------


roc_auc_score:
50.0%
-----------------------------------------------------

/home/guess/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/guess/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [29]:
print(f'Accuracy = {round((cm[0][0]+cm[1][1])/(cm.sum())/.01,1)}%')

Accuracy = 72.4%


In [ ]:
def dropout_classifier_set():
    classifier = Sequential()
    classifier.add(Dense(units=5, kernel_initializer='uniform',
                         activation='relu', input_dim=5))
    classifier.add(Dropout(rate=0.12))
    classifier.add(
        Dense(units=40, kernel_initializer='uniform', activation='relu'))
    classifier.add(Dropout(rate=0.08))
    classifier.add(
        Dense(units=8, kernel_initializer='uniform', activation='relu'))
    classifier.add(Dropout(rate=0.04))
    classifier.add(
        Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))

    classifier.compile(
        optimizer='Nadam', loss='binary_crossentropy', metrics=['accuracy'])
    return classifier

In [ ]:
# classifier = dropout_classifier()

In [ ]:
classifier.fit(X_train, y_train, batch_size=13, 
                   epochs=250, shuffle=False)

In [ ]:
batch_list = [134, 67, 28, 14, 7, 4]
for num in batch_list:
    classifier.fit(X_train, y_train, batch_size=num, 
                   epochs=100, shuffle=False)

In [ ]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

In [ ]:
cm = confusion_matrix(y_test, y_pred)
cm

In [ ]:
print(f'Accuracy = {(cm[0][0]+cm[1][1])/(cm.sum())/.01}%')

### 75.5% Accuracy on 1824 training, 200 test
__3 layers__
- layer 1 = ReLU X 7
- layer 2 = SoftMax X 14
- layer 3 = ReLU X 7
- output layer = Sigmoid X 1

__metrics__ = Cosine

__loss__ = binary_crossentropy

__Optimizer__ = Nadam

__Batch size__ = 12

__Epochs__ = 175

no __diff__

In [ ]:
classifier = KerasClassifier(build_fn=_classifier, batch_size=12, epochs=200)

In [ ]:
accus = cross_val_score(estimator=classifier, X=X_train,
                        y=y_train, cv=10, n_jobs=-2, scoring='accuracy')
accus

In [ ]:
accu_mean = accus.mean()
accu_mean

In [ ]:
accu_var = accus.std()
accu_var

In [ ]:
accus

In [ ]:
def dropout_classifier(optimizer, metrics):
    classifier = Sequential()
    classifier.add(Dense(units=5, kernel_initializer='uniform',
                         activation='relu', input_dim=5))
    classifier.add(Dropout(rate=0.12))
    classifier.add(
        Dense(units=40, kernel_initializer='uniform', activation='relu'))
    classifier.add(Dropout(rate=0.08))
    classifier.add(
        Dense(units=8, kernel_initializer='uniform', activation='relu'))
    classifier.add(Dropout(rate=0.04))
    classifier.add(
        Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))

    classifier.compile(
        optimizer=optimizer, loss='binary_crossentropy', metrics=[metrics])
    return classifier